In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
#get base model
base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
    if freeze_all:
        for layer in model.layers:
            model.trainable = False
    elif (freeze_till is not None) and (freeze_till > 0):
        for layer in model.layers[:-freeze_till]:
            model.trainable = False

    flatten_in = tf.keras.layers.Flatten()(model.output)
    prediction = tf.keras.layers.Dense(
        units = classes,
        activation = "softmax"
    )(flatten_in)

    full_model = tf.keras.models.Model(
        inputs = model.input,
        outputs = prediction
    )

    full_model.compile(
        optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate),
        loss = tf.keras.losses.CategoricalCrossentropy(),
        metrics = ["accuracy"]
    )

    full_model.summary()
    return full_model

In [6]:
#prepare full model
full_model = prepare_full_model(model=base_model, classes=22, freeze_all=True, freeze_till=None,learning_rate=0.01) #freeze all for now
full_model

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 22)                  │         551,958 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 15,266,646 (58.24 MB)

 Trainable params: 551,958 (2.11 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

<Functional name=functional, built=True>

In [7]:
################

In [8]:
#callbacks
import time
import os

In [9]:
def create_tb_callbacks():
    timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    tb_running_log_dir = os.path.join(
        "/content/callbacks/tensorboard_log_dir",
        f"tb_logs_at_{timestamp}"
    )

    return tf.keras.callbacks.TensorBoard(
            log_dir=tb_running_log_dir,
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            write_steps_per_second=False,
            update_freq='epoch',
            profile_batch=0,
            embeddings_freq=0,
            embeddings_metadata=None
        )

def create_ckpt_callbacks():
    # timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    # tb_running_log_dir = os.path.join(
    #     "/content/callbacks/model.keras",
    #     f"tb_logs_at_{timestamp}"
    # )

    return tf.keras.callbacks.ModelCheckpoint(
          filepath = "/content/callbacks/checkpoint_dir/model.keras",
          monitor='val_loss',
          verbose=0,
          save_best_only=False,
          save_weights_only=False,
          mode='auto',
          save_freq='epoch',
          initial_value_threshold=None
    )

def combine_list_of_callbacks():
    return [create_tb_callbacks(), create_ckpt_callbacks()]

In [10]:
###############

In [11]:
params_image_size = [224,224,3]
params_batch_size = 16
training_data = "/content/drive/MyDrive/FitIn/FitIn_classes"
is_augmentation = True

def training_configuration():
    datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

    dataflow_kwargs = dict(
            target_size = params_image_size[:-1],
            batch_size = params_batch_size,
            interpolation = "bilinear"
        )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

    valid_generator = valid_datagenerator.flow_from_directory(
          directory = training_data,
          subset = "validation",
          shuffle = False,
          **dataflow_kwargs
      )

    if is_augmentation:
        train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
    else:
        train_datagenerator = valid_datagenerator

    train_generator = train_datagenerator.flow_from_directory(
            directory = training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    return train_generator, valid_generator

def initiate_train(callback_list:list):
    train_generator, valid_generator = training_configuration()
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = valid_generator.samples // train_generator.batch_size

    full_model.fit(
        train_generator,
        epochs = 30,
        steps_per_epoch = steps_per_epoch,
        validation_steps = validation_steps,
        validation_data = valid_generator,
        callbacks = callback_list
    )



In [12]:
callback_list = combine_list_of_callbacks()


early_stopping = EarlyStopping(
    monitor='val_loss',          # Monitor validation loss
    patience=3,                  # Stop training if no improvement for 5 epochs
    restore_best_weights=True,   # Restore the weights of the best epoch
    verbose=1                    # Print a message when stopping
)
initiate_train(callback_list)

Found 1147 images belonging to 22 classes.
Found 4606 images belonging to 22 classes.
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


287/287 ━━━━━━━━━━━━━━━━━━━━ 1457s 5s/step - accuracy: 0.1648 - loss: 20.7833 - val_accuracy: 0.2694 - val_loss: 8.2278
Epoch 2/30
  1/287 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - accuracy: 0.2500 - loss: 7.0821

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.2500 - loss: 7.0821 - val_accuracy: 0.0909 - val_loss: 4.2088
Epoch 3/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 97s 331ms/step - accuracy: 0.4480 - loss: 6.1857 - val_accuracy: 0.3935 - val_loss: 7.2039
Epoch 4/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5625 - loss: 4.9832 - val_accuracy: 0.1818 - val_loss: 1.6986
Epoch 5/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 97s 330ms/step - accuracy: 0.5527 - loss: 3.9137 - val_accuracy: 0.4261 - val_loss: 4.0731
Epoch 6/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7500 - loss: 2.1317 - val_accuracy: 0.7273 - val_loss: 0.6860
Epoch 7/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 98s 333ms/step - accuracy: 0.6345 - loss: 2.9373 - val_accuracy: 0.5246 - val_loss: 3.9166
Epoch 8/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6250 - loss: 1.7867 - val_accuracy: 1.0000 - val_loss: 1.1921e-04
Epoch 9/30
287/287 ━━━━━━━━━━━━━━━━━━━━ 97s 330ms/step - accuracy: 0.7059 - loss: 1.8607 - val_accur

In [14]:
training_data = "/content/drive/MyDrive/FitIn/FitIn_classes"
def valid_generator():
    datagenerator_kwargs = dict(
        rescale = 1./255,
        validation_split = 0.20
    )

    dataflow_kwargs = dict(
        target_size = [224,224,3][:-1],
        batch_size = 16,
        interpolation = "bilinear"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    valid_generator_param = valid_datagenerator.flow_from_directory(
        directory = training_data,
        subset = "validation",
        shuffle = False,
        **dataflow_kwargs
    )
    return valid_generator_param

def initiate_evaluation():
    valid_generator_param = valid_generator()
    score = full_model.evaluate(valid_generator_param)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [15]:
initiate_evaluation()

Found 1147 images belonging to 22 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 123ms/step - accuracy: 0.4167 - loss: 4.8924
Test loss: 4.539607524871826
Test accuracy: 0.47340890765190125


In [ ]:

full_model.save("/content/saved_model")